# Class Methods & Properties

### Class Attributes

Sometimes we want to share data between all instances of a given class.

All cars have 4 wheels, so we could define a shared variable accessible to all instances of the `Car` class.

To do this, we create them within the `class` body, usually right above the `__init__`.

In [26]:
import datetime

class Car:
    # class attribute
    wheels = 4
    registrations = []

    def __init__(self, make, model, year):
        self.make = make 
        self.model = model 
        self.year = year
        #self.wheels = 0
        Car.registrations.append(self)
        #self.registrations = []
    
    def compute_age(self):
        return datetime.date.today().year - self.year 
    
    
car1 = Car("Honda", "Accord", 2019)
car2 = Car("Toyota", "RAV4", 2006)

In [27]:
# class attribute can be accessed on instances, or the class itself

print(Car.wheels)
print(car1.wheels)
print(car2.wheels)

4
4
4


In [28]:
# these are all the same variable
Car.wheels is car1.wheels

True

In [4]:
# which means changes to the class attribute
# will modify for all classes 

Car.wheels = 3
print(car1.wheels)
print(car2.wheels)

3
3


In [31]:
# note: assigning to an instance attribute makes a new attribute

# creates a new instance variable!
car2.wheels = 2
print(car2.wheels is car1.wheels)
print(car1.wheels)
print(Car.wheels)

False
4
4


### Class Methods

It can also be useful to provide methods that are accessible to all instances of a class.

Class methods are similar to instance methods with a few distinctions:

1. They can not access instance methods or attributes.
2. The first argument to the method is not `self`, but instead `cls` by convention.  `cls` is the class object itself (e.g. `Car`)
3. Class methods are declared with the `@classmethod` decorator.

In [32]:
from datetime import date

class Car: 
    
    # wheels class attribute 
    wheels = 4
    # tire pressure class attribute  
    psi = 35 
    
    def __init__(self, make, model, year):
        self.make = make 
        self.model = model 
        self.year = year
    
    def compute_age(self):
        print(self)
        current_year = int(date.today().year)
        return current_year - self.year 
    
    @classmethod 
    def tire_description(cls):
        print(cls)
        return f'Car has {cls.wheels} wheels with a tire pressure of {Car.psi}' 
    
car1 = Car("Honda", "Accord", 2019)
car2 = Car("Toyota", "RAV4", 2006)

In [35]:
print(Car.tire_description())
#print(car1.tire_description())
print(car1.compute_age())

<class '__main__.Car'>
Car has 4 wheels with a tire pressure of 35
5


Notice that we can use `Car.psi` or `cls.wheels` to access class attributes. `cls` is generally preferred, both to avoid repetition and for reasons we'll see when we get to inheritance.

Finally, note that we can access class methods and instances from within instance methods. (but not vice-versa!)

In [8]:
from datetime import date
class Car: 
    
    # wheels class attribute 
    wheels = 4
    
    # tire pressure amount 
    psi = 35 
    
    def __init__(self, make, model, year):
        self.make = make 
        self.model = model 
        self.year = year
    
    def compute_age(self):
        current_year = int(date.today().year)
        return current_year - self.year 
    
    @classmethod 
    def tire_description(cls):
        return f'Car has {cls.wheels} wheels, each with a tire pressure of {Car.psi}' 

    def __repr__(self): 
        instance_str = f'Car(make={self.make}, model={self.model}, year={self.year}, '
        instance_str += f'wheels={Car.wheels}, {self.tire_description()})'
        return instance_str

In [9]:
car1 = Car("Honda", "Civic", 2019)
print(car1)

Car(make=Honda, model=Civic, year=2019, wheels=4, Car has 4 wheels, each with a tire pressure of 35)


### Alternate Constructors

A common use of class methods is to define alternate ways to initialize an isntance.  In Python there can only be one constructor (`__init__`), whereas some other languages allow multiple.

Perhaps we have Car data coming from a file, meaning we'd have strings like:

In [40]:
car1str = "Pontiac|Grand Am|1997|4892"
car2str = "Ford|Mustang|1970|800"
car3str = "Hyundai|Sonata|2007|0"


def make_car_from_string(s: str) -> Car:
    ...

In [41]:
from datetime import date

class Car: 
    wheels = 4
    psi = 35
    
    def __init__(self, make, model, year):
        self.make = make 
        self.model = model 
        self.year = year
        self.mileage = 0
        
    @classmethod
    def from_string(cls, string):
        make, model, year, mileage = string.split("|")
        # invoke Car's constructor
        new_instance = cls(make, model, year)
        new_instance.mileage = mileage
        return new_instance
    
    def compute_age(self):
        current_year = int(date.today().year)
        return current_year - self.year 
    
    @classmethod 
    def tire_description(cls):
        return f'Car has {cls.wheels} wheels, each with a tire pressure of {Car.psi}' 

    def __repr__(self): 
        instance_str = f'Car(make={self.make}, model={self.model}, year={self.year}, '
        instance_str += f'wheels={Car.wheels})'
        return instance_str

In [42]:
car1 = Car.from_string(car1str)
car2 = Car.from_string(car2str)
car3 = Car.from_string(car3str)

In [43]:
print(car1)
print(car2)
print(car3)

Car(make=Pontiac, model=Grand Am, year=1997, wheels=4)
Car(make=Ford, model=Mustang, year=1970, wheels=4)
Car(make=Hyundai, model=Sonata, year=2007, wheels=4)


This is a common pattern, seen throughout Python:

 - ``int.from_bytes()``
 - ``float.fromhex()`` 
 - ``datetime.date.fromtimestamp()``
 - ``itertools.chain.from_iterable()``


In [45]:
x = list(map(...))
y = dict(...)

TypeError: map() must have at least two arguments.

In [13]:
import datetime
datetime.date(2024, 11, 11)

datetime.date(2024, 11, 11)

In [14]:
datetime.date.fromtimestamp(1234567890)

datetime.date(2009, 2, 13)

In [46]:
import itertools
for x in itertools.chain.from_iterable([(1,2,3), (4,5,6)]):
    print(x)
#for x in (1,2,3):
#    print(x)
#for x in (4,5,6):
#    print(x)

1
2
3
4
5
6


### staticmethod

Sometimes it makes sense to just attach a method to a class for the purpose of namespacing.

In [17]:
def which_is_newer(a, b):
    if a.year > b.year:
        return a
    else:
        return b

which_is_newer(car1, car2)

Car(make=Pontiac, model=Grand Am, year=1997, wheels=4)

In [18]:
# it might make sense to attach this to the class, 
# but neither a classmethod nor an instance method

from datetime import date
class Car: 
    wheels = 4
    psi = 35
    
    # does not take self or cls
    @staticmethod
    def which_is_newer(a, b):
        if a.year > b.year:
            return a
        else:
            return b
        
    @staticmethod
    def something():
        return []
    

    
    def __init__(self, make, model, year):
        self.make = make 
        self.model = model 
        self.year = year
        
    @classmethod
    def from_string(cls, string):
        make, model, year = string.split("|")
        # invoke Car's constructor
        return cls(make, model, year)

    def __repr__(self): 
        instance_str = f'Car(make={self.make}, model={self.model}, year={self.year}, '
        instance_str += f'wheels={Car.wheels})'
        return instance_str

In [19]:
# now would be called this way
Car.which_is_newer(car1, car2)

Car(make=Pontiac, model=Grand Am, year=1997, wheels=4)

### Encapsulation

>``[Encapsulation] allows the implementation of an object's interface to be changed without impacting the users of that object."

The main idea of encapsulation is to hide implementation details from the users of an object. You only expose a public interface to the users.

There are a few ways to encapsulation is handled in Python: 

- Private attributes using underscores
- Getter/Setters
- Properties

### Private Attributes

We saw last week, if we define class attributes with double underscores they are not accessible outside the class.

In [20]:
class Example:
    def __init__(self, x, y, z):
        self.x = x
        self._y = y
        self.__z = z
        
    def __repr__(self):
        return f"Example({self.x}, {self._y}, {self.__z})"

instance = Example(1, 2, 3)

In [21]:
# normal public attribute
instance.x

1

In [22]:
# single underscore attributes are private by convention only
# (there is no enforcement)
instance._y

2

In [23]:
# double underscore methods are name-mangled
instance.__z

AttributeError: 'Example' object has no attribute '__z'

### Getters / Setters

Another common pattern to hide data in OOP languages is to use getter and setter methods that control access.

In [2]:
class Person:
    def __init__(self, name, age):
        self.__name = name  #  Assume it has getter/setters 
        self.set_age(age)

    def _calculate_age_from_birthday():
        pass

    def get_age(self):
        return self._calculate_age_from_birthday()

    def set_age(self, age):
        if age < 0:
            raise ValueError("Person can't have a negative age!")
        self.__age = age
        
    def set_name(self, name):
        if " " not in name:
            raise ValueError("must be at least two words")
        self.__name = name

In [3]:
p = Person("C. Montgomery Burns", 100)

In [4]:
p.get_age()

100

In [6]:
p.set_age(101)

In [7]:
p.get_age()

101

In [8]:
p.set_age(-1)

ValueError: Person can't have a negative age!

In [9]:
p.get_age()

101

This can become very tedious, and as we've seen they don't actually protect access to variables.  Therefore we typically **avoid getters and setters in Python.**

### Properties

We want the advantages of encapsulation (being able to avoid improper use, hiding our internal representation, etc.) but without the need to start with a bunch of getter/setter functions from the get go.

Python has a much nicer way to control access to attributes via **properties**.

There is a built in function `property()` that creates and returns a property object.

`property(fget=None, fset=None, fdel=None, doc=None)`

- `fget` is a function to get value of the attribute
- `fset` is a function to set value of the attribute
- `fdel` is a function to delete the attribute
- `doc` is a docstring for the attribute

In [16]:
class Person:
    
    def __init__(self, name, age):
        self.name = name  #  Assume it has getter/setters 
        self.age = age

    def _get_age(self):
        print("inside get age")
        return self.__age

    def _set_age(self, age):
        if age < 0:
            raise ValueError("Person can't have a negative age!")
        self.__age = age
        
    def __repr__(self):
        return f"Person({self.__name!r}, {self.__age})"
        
    age = property(_get_age, _set_age, doc="age of the person")

In [17]:
p = Person("Wayne", 30)
p.age

inside get age


1000

In [15]:
p.age = -1

ValueError: Person can't have a negative age!

In [13]:
print(p.age)

inside get age
30


#### @property

We can also use `property` as a decorator. 

- Place the `@property` directly above the function header of the getter function.

- Place the code `@name_of_property.setter` above the function header of the setter function. You need to replace the name_of_property with the actual name of the property.

- The function names for both the setter/getter need to match.

In [59]:
class Person:
    def __init__(self, name, age):
        self.__name = name  #  Assume it has getter/setters 
        # invokes setter
        self.age = age #self.set_age(age)
        self.birth_date = ...

    @property
    def age(self):
        """ returns the age property """
        print('getter called')
        return self.__age
    #age = property(age)
    
    @age.setter
    def age(self, age):
        print('setter called')
        if age < 0:
            raise ValueError("Person can't have a negative age!")
        self.__age = age
        
    def __repr__(self):
        return f"Person({self.__name!r}, {self.__age})"

In [61]:
p2 = Person("Emma", 28)
#p2.age = -1
print(p2.age)

setter called
getter called
28


This allows us to start class attributes as public, and add properties as needed.

In [37]:
class Point:
    def __init__(self, x, y):
        self.x = x 
        self.y = y

In [38]:
p = Point(10, 10)

#### Read-only/Calculated Properties

In [26]:
class Rectangle: 
    
    def __init__(self,width,height):
        self.width = width 
        self.height = height 
        self.area = width*height
    
    # read-only calculated property
    #@property 
    #def area(self):
    #    return self.width * self.height 

In [29]:
r = Rectangle(3, 9)

In [30]:
print(r.area)

27


In [31]:
# area is dynamically calculated each call
r.width = 6
print(r.area)

27


In [78]:
# but can't be set
r.area = 4

AttributeError: can't set attribute 'area'

In [25]:
del r.area

AttributeError: can't delete attribute 'area'